In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision import transforms as T
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import numpy as np

# ----------------- Config -----------------
BEST_MODEL_PATH = 'best_model_denorm.pth'
DATA_ROOT = ''   # e.g. '/path/to/val'
VAL_CSV   = os.path.join(DATA_ROOT, 'labels_val_updated.csv')
IMG_DIR   = os.path.join(DATA_ROOT, 'images_val')
OUTPUT_CSV = 'predictions_val_latlong.csv'

# ----------------- Dataset -----------------
class GeoCoordDataset(torch.utils.data.Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.img_dir, row.filename)).convert('RGB')
        img = self.transform(img)
        # return image, true lat, true lon, and index
        return img, float(row.latitude), float(row.longitude), idx

# ----------------- Model Definition -----------------
class CoordModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Load backbone without pretrained head
        self.backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14', pretrained=False)
        feat_dim = self.backbone.embed_dim
        # Simple regression head
        self.head = torch.nn.Sequential(
            torch.nn.Linear(feat_dim, feat_dim//2),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(feat_dim//2, 2),
        )

    def forward(self, x):
        f = self.backbone(x)
        return self.head(f)

# ----------------- Prepare Data -----------------
# Read validation CSV and drop missing entries
val_df = pd.read_csv(VAL_CSV).dropna(subset=['latitude','longitude'])
# Compute normalization stats from train split (or reuse stored values)
lat_mean, lat_std = val_df['latitude'].mean(), val_df['latitude'].std()
lon_mean, lon_std = val_df['longitude'].mean(), val_df['longitude'].std()
# Define transforms matching training
IMG_SIZE = 476
val_tf = T.Compose([
    T.Resize(IMG_SIZE+20),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
# Instantiate dataset and dataloader
val_ds = GeoCoordDataset(val_df, IMG_DIR, val_tf)
val_loader = DataLoader(
    val_ds, batch_size=16, shuffle=False,
    num_workers=0, pin_memory=True
)

# ----------------- Load Model -----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CoordModel().to(device)
# Load saved weights
state = torch.load(BEST_MODEL_PATH, map_location=device)
model.load_state_dict(state)
model.eval()

# ----------------- Predict & Evaluate -----------------
preds = []
truths = []
with torch.no_grad():
    for imgs, lat_t, lon_t, idxs in tqdm(val_loader, desc='Predicting'):
        imgs = imgs.to(device)
        out = model(imgs)
        # Denormalize predictions
        lat_p = out[:,0].cpu().numpy() * lat_std + lat_mean
        lon_p = out[:,1].cpu().numpy() * lon_std + lon_mean
        for idx, lp, lop, lt, lot in zip(idxs, lat_p, lon_p, lat_t, lon_t):
            preds.append((int(idx), float(lp), float(lop)))
            truths.append((lt, lot))
# Sort by id and save
preds.sort(key=lambda x: x[0])
df_out = pd.DataFrame(preds, columns=['id','Latitude','Longitude'])
df_out.to_csv(OUTPUT_CSV, index=False)
print(f'Saved predictions to {OUTPUT_CSV}')
# Compute validation MSE (denormalized)
y_true = np.array(truths)
y_pred = df_out[['Latitude','Longitude']].values
mse = mean_squared_error(y_true, y_pred)
print(f'Validation MSE (denorm): {mse:.4f}')


Using cache found in C:\Users\91997/.cache\torch\hub\facebookresearch_dinov2_main
Predicting: 100%|██████████| 23/23 [00:26<00:00,  1.15s/it]

Saved predictions to predictions_val_latlong.csv
Validation MSE (denorm): 169024.5488
